## <B>Calibrate EAR data</B>

In [ ]:
import pandas as pd

# 파일 이름에 사용할 변수
name = "37"

# 영화 이름 리스트
movies = ["기생충1", "기생충2", "낭닥1", "낭닥2", "코1", "코2", "마녀1", "마녀2", "레버1", "레버2"]

# 파일별 데이터 처리
for movie in movies:
    # 파일 경로 생성
    file = f'{name} {movie} 보정.csv'
    
    # 파일 읽기
    data = pd.read_csv(file)
    
    # Timestamp를 초 단위로 변환
    def convert_to_seconds(timestamp):
        minutes, seconds = map(float, timestamp.split(":"))
        return minutes * 60 + seconds

    data['Seconds'] = data['Timestamp'].apply(convert_to_seconds)
    
    # 0.1초 간격으로 그룹화
    data['Time_Group'] = (data['Seconds'] * 10).astype(int) / 10
    
    # 0.1초 간격으로 EAR 평균 계산
    result = data.groupby('Time_Group')['EAR'].mean().reset_index()
    
    # 결과 저장
    output_file = f'{name} {movie} 평균_EAR.csv'
    result.to_csv(output_file, index=False)
    print(f"Processed: {output_file}")

## <B>EAR data calibration (for special cases of time data)</B>

In [ ]:
import pandas as pd

# 파일 이름
file = "37 낭닥1 보정.csv"

# 초 단위로 변환 함수 (첫 번째 시간 기준 자동 계산)
def convert_to_seconds_with_auto_start(timestamp, initial_time):
    initial_minutes, initial_seconds = map(float, initial_time.split(":"))
    initial_total_seconds = initial_minutes * 60 + initial_seconds

    current_minutes, current_seconds = map(float, timestamp.split(":"))
    current_total_seconds = current_minutes * 60 + current_seconds

    # 경계를 넘는 경우 처리 (current_total_seconds < initial_total_seconds)
    if current_total_seconds < initial_total_seconds % 3600:
        current_total_seconds += 3600  # 1시간 추가

    return current_total_seconds

# 파일 읽기
data = pd.read_csv(file)

# 데이터가 비어있는 경우 예외 처리
if data.empty:
    print(f"Empty file: {file}")
else:
    # 첫 번째 시간 값을 기준으로 초기 시간 설정
    initial_time = data['Timestamp'].iloc[0]

    # 'Seconds' 열 추가
    data['Seconds'] = data['Timestamp'].apply(lambda t: convert_to_seconds_with_auto_start(t, initial_time))

    # 0.1초 간격으로 그룹화
    data['Time_Group'] = (data['Seconds'] * 10).astype(int) / 10

    # 0.1초 간격으로 EAR 평균 계산
    result = data.groupby('Time_Group')['EAR'].mean().reset_index()

    # 결과 저장
    output_file = "37 낭닥1 평균_EAR.csv"
    result.to_csv(output_file, index=False)
    print(f"Processed: {output_file}")

## <B>Eye Openness Ratio During Gory Video Viewing Analysis</B>

In [ ]:
import pandas as pd

name = "37"

movies = ["기생충", "낭닥", "코", "마녀", "레버"]

for movie in movies:
    data1 = pd.read_csv(f'{name} {movie}1 평균_EAR.csv')
    data2 = pd.read_csv(f'{name} {movie}2 평균_EAR.csv')

    count_data3 = data1['EAR'].mean()
    count_data4 = data2['EAR'].mean()

    max_data1 = data1['EAR'].max()
    min_data1 = data1['EAR'].min()
    
    max_data2 = data2['EAR'].max()
    min_data2 = data2['EAR'].min()

    # EAR 정규화
    normalized_data1 = (data1['EAR'] / max_data1) * 100
    normalized_data2 = (data2['EAR'] / max_data2) * 100

    normalized_mean_data1 = normalized_data1.mean()
    normalized_mean_data2 = normalized_data2.mean()

    print(f"데이터1.csv의 EAR 평균: {count_data3}")
    print(f"데이터2.csv의 EAR 평균: {count_data4}")
    print(f"데이터1.csv의 EAR 최대값: {max_data1}")
    print(f"데이터1.csv의 EAR 최소값: {min_data1}")
    print(f"데이터2.csv의 EAR 최대값: {max_data2}")
    print(f"데이터2.csv의 EAR 최소값: {min_data2}")
    print(f"정규화된 데이터1.csv의 EAR 평균: {normalized_mean_data1}")
    print(f"정규화된 데이터2.csv의 EAR 평균: {normalized_mean_data2}")
    print("---------------------------------------------------------")

## <B>Blink Frequency Analysis</B>

In [ ]:
import pandas as pd

name = "37"

movies = ["기생충", "낭닥", "코", "마녀", "레버"]

for movie in movies:
    data1 = pd.read_csv(f'{name} {movie}1 평균_EAR.csv')
    data2 = pd.read_csv(f'{name} {movie}2 평균_EAR.csv')

    # EAR 정규화
    max_data1 = data1['EAR'].max()
    min_data1 = data1['EAR'].min()
    
    max_data2 = data2['EAR'].max()
    min_data2 = data2['EAR'].min()

    data1['Normalized EAR'] = ((data1['EAR'] - min_data1) / (max_data1 - min_data1)) * 100
    data2['Normalized EAR'] = ((data2['EAR'] - min_data2) / (max_data2 - min_data2)) * 100

    data1['EAR 0/1'] = (data1['EAR'] >= 0.25).astype(int)
    data2['EAR 0/1'] = (data2['EAR'] >= 0.25).astype(int)

    # 0에서 1로 바뀌는 순간 카운트
    data1['Transition Count'] = ((data1['EAR 0/1'].shift(1) == 0) & (data1['EAR 0/1'] == 1)).astype(int)
    data2['Transition Count'] = ((data2['EAR 0/1'].shift(1) == 0) & (data2['EAR 0/1'] == 1)).astype(int)

    count_transitions_data1 = data1['Transition Count'].sum()
    count_transitions_data2 = data2['Transition Count'].sum()

    data1.to_csv(f'{name} {movie}1 보정_수정.csv', index=False)
    data2.to_csv(f'{name} {movie}2 보정_수정.csv', index=False)

    print(f"데이터1.csv의 0에서 1로 바뀌는 순간 카운트: {count_transitions_data1}")
    print(f"데이터2.csv의 0에서 1로 바뀌는 순간 카운트: {count_transitions_data2}")
    print("---------------------------------------------------------")